In [3]:
%pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple blpapi

Looking in indexes: https://blpapi.bloomberg.com/repository/releases/python/simple
     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/4.7 MB 871.5 kB/s eta 0:00:06
      --------------------------------------- 0.1/4.7 MB 1.3 MB/s eta 0:00:04
     -- ------------------------------------- 0.2/4.7 MB 1.7 MB/s eta 0:00:03
     -- ------------------------------------- 0.3/4.7 MB 1.6 MB/s eta 0:00:03
     ----- ---------------------------------- 0.7/4.7 MB 2.7 MB/s eta 0:00:02
     ------- -------------------------------- 0.9/4.7 MB 3.2 MB/s eta 0:00:02
     --------- ------------------------------ 1.2/4.7 MB 3.3 MB/s eta 0:00:02
     ------------- -------------------------- 1.6/4.7 MB 4.2 MB/s eta 0:00:01
     ---------------- ----------------------- 1.9/4.7 MB 4.4 MB/s eta 0:00:01


In [4]:

import blpapi
options = blpapi.SessionOptions()
options.setServerHost('localhost')
options.setServerPort(8194)
session = blpapi.Session(options)
session.start()

False

In [7]:
import blpapi
import sys


SESSION_STARTED         = blpapi.Name("SessionStarted")
SESSION_STARTUP_FAILURE = blpapi.Name("SessionStartupFailure")
SERVICE_OPENED          = blpapi.Name("ServiceOpened")
SERVICE_OPEN_FAILURE    = blpapi.Name("ServiceOpenFailure")
ERROR_INFO              = blpapi.Name("ErrorInfo")
MANUAL_FILL             = blpapi.Name("ManualFill")

d_service="//blp/emapisvc_beta"
d_host="localhost"
d_port=8194
bEnd=False

class SessionEventHandler():

    def processEvent(self, event, session):
        try:
            if event.eventType() == blpapi.Event.SESSION_STATUS:
                self.processSessionStatusEvent(event,session)
            
            elif event.eventType() == blpapi.Event.SERVICE_STATUS:
                self.processServiceStatusEvent(event,session)

            elif event.eventType() == blpapi.Event.RESPONSE:
                self.processResponseEvent(event)
            
            else:
                self.processMiscEvents(event)
                
        except:
            print ("Exception:  %s" % sys.exc_info()[0])
            
        return False


    def processSessionStatusEvent(self,event,session):  
        print ("Processing SESSION_STATUS event")

        for msg in event:
            if msg.messageType() == SESSION_STARTED:
                print ("Session started...")
                session.openServiceAsync(d_service)
                
            elif msg.messageType() == SESSION_STARTUP_FAILURE:
                print >> sys.stderr, ("Error: Session startup failed")
                
            else:
                print (msg)
                

    def processServiceStatusEvent(self,event,session):
        print ("Processing SERVICE_STATUS event")
        
        for msg in event:
            
            if msg.messageType() == SERVICE_OPENED:
                print ("Service opened...")

                service = session.getService(d_service)
    
                request = service.createRequest("ManualFill");
                #request.set("EMSX_REQUEST_SEQ", 1)
                #request.set("EMSX_TRADER_UUID", 1234567) # Trader UUID

                routeToFill = request.getElement("ROUTE_TO_FILL")
                routeToFill.setElement("EMSX_SEQUENCE", 12345) # EMSX_SEQUENCE or Order# from EMSX blotter
                routeToFill.setElement("EMSX_ROUTE_ID", 1)
                    
                fills = request.getElement("FILLS")

                fill = fills.appendElement()
                    
                fill.setElement("EMSX_FILL_AMOUNT", 100)
                fill.setElement("EMSX_FILL_PRICE", 168.11)
                #fill.setElement("EMSX_LAST_MARKET", "XLON")
                #fills.setElement("EMSX_INDIA_EXCHANGE","BGL")
                fillDateTime = fill.getElement("EMSX_FILL_DATE_TIME")
                    
                legacy = fillDateTime.setChoice("Legacy");
                legacy.setElement("EMSX_FILL_DATE",20171103)
                legacy.setElement("EMSX_FILL_TIME",26070)
                legacy.setElement("EMSX_FILL_TIME_FORMAT","SecondsFromMidnight")

                print ("Request: %s" % request.toString())
    
                self.requestID = blpapi.CorrelationId()
                
                session.sendRequest(request, correlationId=self.requestID )
                            
            elif msg.messageType() == SERVICE_OPEN_FAILURE:
                print >> sys.stderr, ("Error: Service failed to open")        
                
    def processResponseEvent(self, event):
        print ("Processing RESPONSE event")
        
        for msg in event:
            
            print ("MESSAGE: %s" % msg.toString())
            print ("CORRELATION ID: %d" % msg.correlationIds()[0].value())


            if msg.correlationIds()[0].value() == self.requestID.value():
                print ("MESSAGE TYPE: %s" % msg.messageType())
                
                if msg.messageType() == ERROR_INFO:
                    errorCode = msg.getElementAsInteger("ERROR_CODE")
                    errorMessage = msg.getElementAsString("ERROR_MESSAGE")
                    print ("ERROR CODE: %d\tERROR MESSAGE: %s" % (errorCode,errorMessage))
                elif msg.messageType() == MANUAL_FILL:
                    fillID = msg.getElementAsInteger("EMSX_FILL_ID")
                    message = msg.getElementAsString("MESSAGE")
                    print ("EMSX_FILL_ID: %d\tMESSAGE: %s" % (fillID,message))

                global bEnd
                bEnd = True
                
    def processMiscEvents(self, event):
        
        print ("Processing " + event.eventType() + " event")
        
        for msg in event:

            print ("MESSAGE: %s" % (msg.tostring()))


def main():
    
    sessionOptions = blpapi.SessionOptions()
    sessionOptions.setServerHost(d_host)
    sessionOptions.setServerPort(d_port)

    print ("Connecting to %s:%d" % (d_host,d_port))

    eventHandler = SessionEventHandler()

    session = blpapi.Session(sessionOptions, eventHandler.processEvent)

    if not session.startAsync():
        print ("Failed to start session.")
        return
    
    global bEnd
    while bEnd==False:
        pass
    
    session.stop()
    
if __name__ == "__main__":
    print ("Bloomberg - EMSX API Sell-Side  - ManualFill")
    try:
        main()
    except KeyboardInterrupt:
        print ("Ctrl+C pressed. Stopping...")


Bloomberg - EMSX API Sell-Side  - ManualFill
Connecting to localhost:8194
Processing SESSION_STATUS event
Exception:  <class 'TypeError'>
Ctrl+C pressed. Stopping...
